# SiRF Reader - Locosys SiRF Binary

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import numpy as np

from math import log10, floor

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class SirfReader(BaseReader):
    '''SBN + SBP files - Locosys SiRF Binary'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)
        
        self.device = None


    def load(self, headerOnly=False, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            if headerOnly == False:
                self.buffer = f.read()
            else:
                self.buffer = f.read(64)

            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readHeader(ignoreChecksums=True)
                if headerOnly == False:
                    self.readData(ignoreChecksums=ignoreChecksums)


    def readHeader(self, ignoreChecksums=False):
        '''Read Locosys header into memory'''

        # Main header content - ignore the checksum because people have been known to patch the username
        messageId, message = self.readMessage(ignoreChecksums=ignoreChecksums)

        # Parse ASCII data, ignoring message[0] which is the SiRF message ID - typically 0xfd but sometimes 0xdf
        splitFields = message[1:].tobytes().decode('ascii').split(',')
        
        # Decode fields that are always present, regardless of the message ID being 0xfd or 0xdf
        self.header['username'] = splitFields[0]
        try:
            self.header['serial'] = int(splitFields[1])
        except Exception:
            # This handles the rare case of an SBP file with the string "unknown,unknown,1,unknown"
            self.header['serial'] = 0
        self.header['frequency'] = 5 if splitFields[2] == '2' else 1
        self.header['firmware'] = splitFields[3]

        # Note that message ID 0xdf has 2 additional fields (maybe 3) which are currently ignored
        # e.g. "GW60USER,168605168,2,V1.3A0926C,0.0h,00,"

        self.device = getDevice(self.header['serial'])


    def readMessage(self, ignoreChecksums=False):
        '''Read SiRF message from buffer'''

        # Check the start token is a0a2
        startToken = self.readUnsigned16BE()
        if startToken != 0xa0a2:
            raise ValueError('Unexpected start token 0x{:x} - expected 0x{:x}'.format(startToken, 0xa0a2))

        # Internal length is sometimes big endian, sometimes little endian!
        length = self.readUnsigned16BE()
        if length > 255:
            length = length >> 8

        # Main message content
        message = self.readBytes(length)
        messageId = message[0]

        # Evaluate checksum
        checksum = self.readUnsigned16BE()
        if ignoreChecksums == False and checksum != sum(message):
            raise ValueError('Checksum difference - {}'.format(checksum - sum(message)))

        # Check the end token is b0b3
        endToken = self.readUnsigned16BE()
        if endToken != 0xb0b3:
            raise ValueError('Unexpected end token 0x{:x} - expected 0x{:x}'.format(endToken, 0xb0b3))

        return messageId, message


    def consumeRawData(self, track):
        '''Consume raw SiRF data - convert to standard data structure'''

        # Convert HDOP from integer to decimal
        track.data['hdop'] = np.divide(self.rawData['hdop'], 5, dtype='float32')

        # Satellite count is already an integer
        track.data['sat'] = (self.rawData['sv_count']).astype('uint8')

        # Copy satellite IDs
        track.data['svids'] = (self.rawData['sv_ids']).astype('uint32')

        # Convert latitude and longitude to decimal
        track.data['lat'] = np.divide(self.rawData['latitude'], 10000000, dtype='float64')
        track.data['lon'] = np.divide(self.rawData['longitude'], 10000000, dtype='float64')

        # Convert elevation from cm to m
        track.data['ele'] = np.divide(self.rawData['altitude_msl'], 100, dtype='float64')

        # Convert SOG from cm/s to m/s
        track.data['sog'] = np.divide(self.rawData['sog'], 100, dtype='float32')

        # Convert COG from integer to decimal
        track.data['cog'] = np.divide(self.rawData['cog'], 100, dtype='float32')

        # Convert rate of climb from cm/s to m/s
        track.data['roc'] = np.divide(self.rawData['climb_rate'], 100, dtype='float32')

In [3]:
def getDevice(serial):
    '''Determine device type from serial number'''

    device = 'Unknown'
    
    if serial > 0:
        serialPrefix = serial // (10 ** floor(log10(serial) - 2))

        if serialPrefix == 100:
            device = 'GT-11'
        elif serialPrefix in [103, 113, 123, 133, 143, 832, 833, 932, 933]:
            device = 'GT-31'
        elif serialPrefix == 155:
            device = 'GW-52'
        elif serialPrefix == 168:
            device = 'GW-60'
        
    return device

## Unit Tests

In [4]:
class TestInit(unittest.TestCase):
    '''Class to test init'''

    def testFilename(self):
        '''Test the filename is as expected'''

        self.assertEqual(sirfReader.filename, 'test.sbn')


    def testHeader(self):
        '''Test the header is as expected'''

        self.assertEqual(sirfReader.header, {})


    def testTracks(self):
        '''Test the tracks are as expected'''

        self.assertEqual(sirfReader.tracks, [])

In [5]:
class TestDevice(unittest.TestCase):
    '''Class to test device identification - GT-31 serials taken from Weymouth Speed Week 2010-2021'''

    def testGt11(self):
        '''Test GT-11'''

        self.assertEqual(getDevice(1001781), 'GT-11')
        self.assertEqual(getDevice(1003198), 'GT-11')


    def testGt31_103(self):
        '''Test GT-31 - 103 prefix'''

        self.assertEqual(getDevice(103200148), 'GT-31')
        self.assertEqual(getDevice(103201606), 'GT-31')


    def testGt31_113(self):
        '''Test GT-31 - 113 prefix'''

        self.assertEqual(getDevice(113200232), 'GT-31')
        self.assertEqual(getDevice(113300153), 'GT-31')


    def testGt31_123(self):
        '''Test GT-31 - 123 prefix'''

        self.assertEqual(getDevice(123200058), 'GT-31')
        self.assertEqual(getDevice(123300129), 'GT-31')


    def testGt31_133(self):
        '''Test GT-31 - 133 prefix'''

        self.assertEqual(getDevice(133200018), 'GT-31')
        self.assertEqual(getDevice(133201596), 'GT-31')


    def testGt31_143(self):
        '''Test GT-31 - 143 prefix'''

        self.assertEqual(getDevice(143200024), 'GT-31')
        self.assertEqual(getDevice(143200213), 'GT-31')


    def testGt31_832(self):
        '''Test GT-31 - 832 prefix'''

        self.assertEqual(getDevice(832000210), 'GT-31')
        self.assertEqual(getDevice(832004857), 'GT-31')


    def testGt31_833(self):
        '''Test GT-31 - 833 prefix'''

        self.assertEqual(getDevice(833000784), 'GT-31')
        self.assertEqual(getDevice(833002174), 'GT-31')


    def testGt31_932(self):
        '''Test GT-31 - 932 prefix'''

        self.assertEqual(getDevice(932000059), 'GT-31')
        self.assertEqual(getDevice(932001502), 'GT-31')


    def testGt31_933(self):
        '''Test GT-31 - 933 prefix'''

        self.assertEqual(getDevice(933000012), 'GT-31')


    def testGw52(self):
        '''Test GW-52'''

        self.assertEqual(getDevice(155800017), 'GW-52')


    def testGw60(self):
        '''Test GW-60'''

        self.assertEqual(getDevice(168601673), 'GW-60')
        self.assertEqual(getDevice(168605168), 'GW-60')

In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    sirfReader = SirfReader('test.sbn')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.006s

OK
